<a href="https://colab.research.google.com/github/EdoGiordani99/Traffic-Light-Detector/blob/main/Predict_Notebook.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Prediction Notebook
<h2>Vision & Perception Project 2021/2022</h2>
<h3>Student: Edoardo Giordani</h3>
<h3>E-mail: giordani.2020434@studenti.uniroma1.it</h3>
This notebook was used to make inferences on the test set and plot the predicted bounding boxes on them.


##Mounting Drive

In [ ]:
from google.colab import drive
drive.mount('/content/drive')
%cd drive/MyDrive/V&P/TLD

Mounted at /content/drive
/content/drive/MyDrive/V&P/TLD


##Libraries Import

In [ ]:
!pip install roboflow
!pip install albumentations==0.4.6
!pip install pytest-shutil

In [ ]:
import os
import cv2
import time
import torch
import pickle
import shutil
import numpy as np
import glob as glob
import albumentations as A

from roboflow import Roboflow
from matplotlib import pyplot as plt
from albumentations.pytorch import ToTensorV2

from model import create_model
from config import (
    NUM_CLASSES, DEVICE, CLASSES, MODEL_NAME, PRETRAINED
)

In [ ]:
DEVICE

device(type='cpu')

## Settings

In [ ]:
TEST_DIR = 'data/test'

WEIGHTS_PATH = 'trainings/1/best_model.pth'

THRESHOLD = 0.7

## Loading the Trained Model

In [ ]:
model = create_model(num_classes=NUM_CLASSES, name=MODEL_NAME, pretrained=PRETRAINED)
checkpoint = torch.load(WEIGHTS_PATH , map_location=DEVICE)
model.load_state_dict(checkpoint['model_state_dict'])
model.to(DEVICE).eval()

print('Model Loaded Correctly')

Downloading: "https://download.pytorch.org/models/fasterrcnn_resnet50_fpn_coco-258fb6c6.pth" to /root/.cache/torch/hub/checkpoints/fasterrcnn_resnet50_fpn_coco-258fb6c6.pth


  0%|          | 0.00/160M [00:00<?, ?B/s]

Model Loaded Correctly


## Making Prediction

In [ ]:
#@title Predict Function

def predict(img):

    # Processing to make the image net - compatible
    img = img.astype(np.float32)
    img /= 255.0
    img = np.transpose(img, (2, 0, 1)).astype(np.float32)

    img = torch.tensor(img, dtype=torch.float).cuda()

    #adding the batch dimension
    img = torch.unsqueeze(img, 0)

    with torch.no_grad():
        outputs = model(img.to(DEVICE))

    outputs = [{k: v.to('cpu') for k, v in t.items()} for t in outputs]

    return outputs

In [ ]:
#@title Plot Functions

def take_color(class_name): 
    if class_name == 'Red Car':
        color = (225, 0, 0)

    elif class_name == 'Red Ped': 
        color = (255, 0, 150)
    
    elif class_name == 'Green Car':
        color = (0, 255, 0)

    elif class_name == 'Green Ped': 
        color = (0, 255, 150)

    elif class_name == 'Yellow Car':
        color = (225, 150, 0)

    elif class_name == 'Yellow Ped': 
        color = (255, 150, 150)
    
    elif class_name == 'Unk': 
        color = (255, 0, 255)

    return color

def visualize_bbox(img, bbox, class_name):

    BOX_COLOR = take_color(class_name) # Red
    TEXT_COLOR = (255, 255, 255) # White

    x_min, x_max, y_min, y_max = bbox
   
    cv2.rectangle(img, (x_min, y_min), (x_max, y_max), color=BOX_COLOR, thickness=12)
    
    ((text_width, text_height), _) = cv2.getTextSize(class_name, cv2.FONT_HERSHEY_SIMPLEX, 4, 2)    
    cv2.rectangle(img, (x_min, y_min - int(1.3 * text_height)), (x_min + text_width, y_min), BOX_COLOR, -1)

    cv2.putText(
        img,
        text=class_name,
        org=(x_min, y_min - int(0.3 * text_height)),
        fontFace=cv2.FONT_HERSHEY_SIMPLEX,
        fontScale=4, 
        color=TEXT_COLOR,
        thickness = 10
    )
    return img


def visualize(image, threshold, outputs):

    if len(outputs[0]['boxes']) != 0:
        # getting the boxes
        boxes = outputs[0]['boxes'].data.numpy()
        # getting the scores
        scores = outputs[0]['scores'].data.numpy()

    # Bounding Boxes Predictions
    boxes = boxes[scores >= threshold].astype(np.int32)
    bboxes = boxes.copy()

    # Class Predictions
    pred_classes = [CLASSES[i] for i in outputs[0]['labels'].cpu().numpy()][:len(boxes)]
    print(pred_classes)

    img = image.copy()
    for bbox, class_name in zip(bboxes, pred_classes):
        img = visualize_bbox(img, bbox, class_name)
    plt.figure(figsize=(12, 12))
    plt.axis('off')
    plt.imshow(img)

In [ ]:
test_images = glob.glob(f"{TEST_DIR}/*.jpg")
print(f"Test instances: {len(test_images)}")

Test instances: 26


In [ ]:
for img in test_images:
    image = cv2.imread(img)
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

    outputs = predict(image)

    visualize(image, THRESHOLD, outputs)

RuntimeError: ignored